In [20]:
import tensorflow as tf

In [1]:
import autokeras as ak

2023-07-23 02:49:13.501536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('data/experimental_df.csv')


In [46]:
X_train = df.drop(columns=['Winner'])
y_train = df['Winner'].astype(int)

In [48]:
clf = ak.StructuredDataClassifier()

INFO:tensorflow:Reloading Tuner from ./structured_data_classifier/tuner0.json


In [49]:
clf.fit(X_train, y_train, epochs=10)


Trial 62 Complete [00h 00m 02s]
val_accuracy: 0.27272728085517883

Best val_accuracy So Far: 0.9090909361839294
Total elapsed time: 00h 02m 28s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
9/9 [==============================] - 0s 3ms/step - loss: 1.3436 - accuracy: 0.4307
Epoch 2/10
9/9 [==============================] - 0s 3ms/step - loss: 1.2618 - accuracy: 0.4120
Epoch 3/10
9/9 [==============================] - 0s 3ms/step - loss: 1.2294 - accuracy: 0.4045
Epoch 4/10
9/9 [==============================] - 0s 3ms/step - loss: 1.2472 - accuracy: 0.3858
Epoch 5/10
9/9 [==============================] - 0s 3ms/step - loss: 1.2012 - accuracy: 0.3745
Epoch 6/10
9/9 [==============================] - 0s 3ms/step - loss: 1.2746 - accuracy: 0.4120
Epoch 7/10
9/9 [==============================] - 0s 3ms/step - loss: 1.2124 - accuracy: 0.3783
Epoch 8/10
9/9 [==============================] - 0s 3ms/step - loss: 1.1964 - accuracy: 0.4195
Epoch 9/10
9/9 [==============================] - 

In [59]:
model = clf.export_model()

In [60]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 38)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 38)               0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 38)               77        
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                624       
                                                                 
 re_lu (ReLU)                (None, 16)                0         
                                                                 
 dropout (Dropout)           (None, 16)                0     

In [61]:
model.save('autokeras_model')

INFO:tensorflow:Assets written to: autokeras_model/assets
